# Functions

In [1]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import wandb
import time
import torch
import torch.nn.functional as F

class HelperClass:
    def __init__(self, model, criterion, optimizer, device):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train(self, train_loader,test_loader, num_epochs):
        self.model.to(self.device)
        self.model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            start_traintime = time.time()
            for i, (data) in enumerate(train_loader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()
            train_loss = running_loss / (i+1)
            endtime_train = time.time() - start_traintime
            self.evaluate(test_loader=test_loader, train_loader=train_loader, train_loss=train_loss, endtime_train=endtime_train)
            print(f"Epoch {epoch+1}, Loss: {train_loss}")
        print("Finished Training")
        wandb.finish()
        return self.model


    def evaluate(self, test_loader, train_loader, train_loss, endtime_train):
        # Evaluate the model on test_loader
        self.model.to(self.device)
        self.model.eval()
        correct = 0
        total = 0
        test_loss = 0
        starttime_test = time.time()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                test_loss += self.criterion(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        test_loss = test_loss / total

        # Evaluate the model on the train_loader
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        endtime_test = time.time() - starttime_test
        wandb.log(
            {
                "test_accuracy": test_accuracy,
                "train_accuracy": train_accuracy,
                "train_loss": train_loss,
                "test_loss": test_loss,
                "time_train": endtime_train,
                "time_test": endtime_test,

            }
        )




def prepare_data(batch_size):
    training_data = datasets.CIFAR10(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )

    test_data = datasets.CIFAR10(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )

    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    return train_dataloader, test_loader

def wandb_login(dict):

    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="del-MC1",

        # track hyperparameters and run metadata
        config=dict
    )
    
def get_run_hist(run_id):
    api = wandb.Api()
    run = api.run(path=f'del-MC1/{run_id}')
    history = run.history()
    return history

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

C:\Users\denis\AppData\Roaming\Python\Python311\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] Die angegebene Prozedur wurde nicht gefunden'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cuda:0


In [2]:
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.device_count())   # Should return the number of GPUs available
print(torch.cuda.get_device_name(0))  # Returns the name of the GPU being used


True
1
NVIDIA GeForce RTX 3050 Ti Laptop GPU


# Load Dataset

In [ ]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

In [ ]:
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# EDA

In [ ]:
import pickle
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
label_names = unpickle("data/cifar-10-batches-py/batches.meta")[b'label_names']
print(label_names)
label_names = [x.decode('utf-8') for x in label_names]
print(label_names)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [ ]:
data_train, labels_train = next(iter(train_dataloader))
print(f"Feature batch shape: {data_train.shape}")
print(f"Labels batch shape: {labels_train.shape}")

In [ ]:
img = data_train[0].squeeze()
label = label_names[labels_train[0]]
# move layer 1 to 3
img = img.permute(1, 2, 0)
plt.imshow(img, cmap="gray")
plt.title(f"Label: {label}")
plt.show()

In [ ]:
# Get all Data from CIFAR-10
data_train = training_data.data
labels_train = training_data.targets
print(training_data.data.shape)

In [ ]:
import numpy as np
index = np.where(np.array(labels_train) == 0)[0]
class_data = data_train[index]
class_data.shape

In [ ]:
# Show ten Pictures per class in CIFAR-10 resulting in 100 pictures
fig, axs = plt.subplots(10, 10, figsize=(15, 15))
for i in range(10):
    index = np.where(np.array(labels_train) == i)[0]
    class_data = data_train[index]
    for j in range(10):
        img = class_data[j].squeeze()
        axs[i, j].imshow(img, cmap="gray")
        axs[i, j].axis('off')
        axs[i, j].set_title(label_names[i])
plt.show()

## Preprocessing
Ich werde die Farbwerte der Bilder zwischen 0 und 1 skalieren.

In [ ]:
plt.imshow(training_data.data[0])
plt.legend()

In [ ]:
#training_data.data = training_data.data / 255

In [ ]:
#test_data.data = test_data.data / 255

In [ ]:
# Control
plt.imshow(training_data.data[0])

# Overfit mit MLP

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


batch_size = 16
learning_rate = 0.1
epochs = 100

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel()
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model_class = HelperClass(baseModel, criterion, optimizer, device)

training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
batch_samples = torch.utils.data.Subset(training_data, range(batch_size))
train_loader = DataLoader(batch_samples, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 2,
            "learning_rate": learning_rate,
            "architecture": "MLP",
            "batch_size": batch_size,
            "conv_layers": 0,
        }

wandb_login(dict)

trained_model = model_class.train(train_loader, test_loader, epochs)

## Analyse

In [ ]:
history = get_run_hist("cu5jzl9b")
plt.plot(history['train_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Train Accuracy')
plt.title('Train Accuracy for Overfitting-Test')
plt.show()

# Einfache MLP-Modelle
## Base Model

In [ ]:

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 720)
        self.fc2 = nn.Linear(720, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


batch_size = 16
learning_rate = 0.01
epochs = 50

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel()
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model_class = HelperClass(baseModel, criterion, optimizer, device)

train_loader, test_loader = HelperClass.prepare_data(batch_size)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 3,
            "learning_rate": learning_rate,
            "architecture": "MLP",
            "batch_size": batch_size,
            "conv_layers": 0,
        }

wandb_login(dict)

trained_model = model_class.train(train_loader, test_loader, epochs)

In [ ]:
history = get_run_hist("6yxvv84m")

In [ ]:
#make same plot but with plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=history['train_accuracy'], mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(y=history['test_accuracy'], mode='lines', name='Test Accuracy'))
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Accuracy')
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, subplot_titles=("Train Loss", "Test Loss"))
fig.add_trace(go.Scatter(y=history['train_loss'], mode='lines', name='Train Loss'), row=1, col=1)
fig.add_trace(go.Scatter(y=history['test_loss'], mode='lines', name='Test Loss'), row=2, col=1)

fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_xaxes(title_text='Epoch', row=2, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=2, col=1)

fig.show()

TODO: Analyse

## Hyperparameter Tuning
### Anzahl Layers MLP
#### 4 Layers

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc3 = nn.Linear(1000, 256)
        self.fc4 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


batch_size = 16
learning_rate = 0.01
epochs = 50

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel()
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model_class = HelperClass(baseModel, criterion, optimizer, device)

train_loader, test_loader = prepare_data(batch_size)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 4,
            "learning_rate": learning_rate,
            "architecture": "MLP",
            "batch_size": batch_size,
            "conv_layers": 0,
        }

wandb_login(dict)

trained_model = model_class.train(train_loader, test_loader, epochs)

#### 2 Layers

In [ ]:
from HelperClasses.HelperClasses import HelperClass
from HelperClasses import HelperClasses as hp

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


batch_size = 16
learning_rate = 0.01
epochs = 50

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel()
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model_class = HelperClass(baseModel, criterion, optimizer, device)

train_loader, test_loader = prepare_data(batch_size)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 2,
            "learning_rate": learning_rate,
            "architecture": "MLP",
            "batch_size": batch_size,
            "conv_layers": 0,
        }

wandb_login(dict)

trained_model = model_class.train(train_loader, test_loader, epochs)

#### 5 Layer

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x


batch_size = 16
learning_rate = 0.01
epochs = 10

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel()
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model_class = HelperClass(baseModel, criterion, optimizer, device)

train_loader, test_loader = hp.prepare_data(batch_size)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 5,
            "learning_rate": learning_rate,
            "architecture": "MLP",
            "batch_size": batch_size,
            "conv_layers": 0,
        }

hp.wandb_login(dict)

trained_model = model_class.train(train_loader, test_loader, epochs)

#### Analyse


In [11]:
hist4layers = get_run_hist("0qsnvr7j")
hist2layers = get_run_hist("zjjwg04a")
hist5layers = get_run_hist("h8iftc74")
histBaseModel = get_run_hist("6yxvv84m")

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define colors for each label
color_map = {
    '5 Layers': 'blue',
    '4 Layers': 'black',
    '2 Layers': 'gray',
    'BaseModel': 'orange'
}

# Create a 2x2 grid of subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=("Train Loss", "Test Loss", "Train Accuracy", "Test Accuracy"))

# Row 1, Col 1 (Train Loss)
fig.add_trace(go.Scatter(y=hist5layers['train_loss'], mode='lines', name='5 Layers', legendgroup='5 Layers', hoverinfo='name+y', line={"color":color_map['5 Layers']}), row=1, col=1)
fig.add_trace(go.Scatter(y=hist4layers['train_loss'], mode='lines', name='4 Layers',  legendgroup='4 Layers', hoverinfo='name+y', line={"color":color_map['4 Layers']}), row=1, col=1)
fig.add_trace(go.Scatter(y=hist2layers['train_loss'], mode='lines', name='2 Layers', legendgroup='2 Layers', hoverinfo='name+y', line={"color":color_map['2 Layers']}), row=1, col=1)
fig.add_trace(go.Scatter(y=histBaseModel['train_loss'], mode='lines', name='BaseModel', legendgroup='BaseModel', hoverinfo='name+y', line={"color":color_map['BaseModel']}), row=1, col=1)

# Row 1, Col 2 (Test Loss)
fig.add_trace(go.Scatter(y=hist5layers['test_loss'], mode='lines', name='5 Layers', legendgroup='5 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['5 Layers']}), row=1, col=2)
fig.add_trace(go.Scatter(y=hist4layers['test_loss'], mode='lines', name='4 Layers', legendgroup='4 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['4 Layers']}), row=1, col=2)
fig.add_trace(go.Scatter(y=hist2layers['test_loss'], mode='lines', name='2 Layers', legendgroup='2 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['2 Layers']}), row=1, col=2)
fig.add_trace(go.Scatter(y=histBaseModel['test_loss'], mode='lines', name='BaseModel', legendgroup='BaseModel', hoverinfo='name+y', showlegend=False, line={"color":color_map['BaseModel']}), row=1, col=2)

# Row 2, Col 1 (Train Accuracy)
fig.add_trace(go.Scatter(y=hist5layers['train_accuracy'], mode='lines', name='5 Layers', legendgroup='5 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['5 Layers']}), row=2, col=1)
fig.add_trace(go.Scatter(y=hist4layers['train_accuracy'], mode='lines', name='4 Layers', legendgroup='4 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['4 Layers']}), row=2, col=1)
fig.add_trace(go.Scatter(y=hist2layers['train_accuracy'], mode='lines', name='2 Layers', legendgroup='2 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['2 Layers']}), row=2, col=1)
fig.add_trace(go.Scatter(y=histBaseModel['train_accuracy'], mode='lines', name='BaseModel', legendgroup='BaseModel', hoverinfo='name+y', showlegend=False, line={"color":color_map['BaseModel']}), row=2, col=1)

# Row 2, Col 2 (Test Accuracy)
fig.add_trace(go.Scatter(y=hist5layers['test_accuracy'], mode='lines', name='5 Layers', legendgroup='5 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['5 Layers']}), row=2, col=2)
fig.add_trace(go.Scatter(y=hist4layers['test_accuracy'], mode='lines', name='4 Layers', legendgroup='4 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['4 Layers']}), row=2, col=2)
fig.add_trace(go.Scatter(y=hist2layers['test_accuracy'], mode='lines', name='2 Layers', legendgroup='2 Layers', hoverinfo='name+y', showlegend=False, line={"color":color_map['2 Layers']}), row=2, col=2)
fig.add_trace(go.Scatter(y=histBaseModel['test_accuracy'], mode='lines', name='BaseModel', legendgroup='BaseModel', hoverinfo='name+y', showlegend=False, line={"color":color_map['BaseModel']}), row=2, col=2)

# Set x-axis and y-axis labels for each subplot
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_xaxes(title_text='Epoch', row=1, col=2)
fig.update_xaxes(title_text='Epoch', row=2, col=1)
fig.update_xaxes(title_text='Epoch', row=2, col=2)

fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=2)
fig.update_yaxes(title_text='Accuracy [%]', row=2, col=1)
fig.update_yaxes(title_text='Accuracy [%]', row=2, col=2)

# Show the figure
fig.show()


TODO: Analyse 

### Layergröße
#### Grosse Layer


#### Kleine Layer


TODO: Analyse

### Learning Rate

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 720)
        self.fc2 = nn.Linear(720, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

batch_size = 16
# Base Model ist 0.01
learning_rates = [0.0001, 0.001, 0.1, 1]
#learning_rates = [0.01]
epochs = 50

for learning_rate in learning_rates:
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer
    baseModel = BaseModel()
    optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)
    
    model_class = HelperClass(baseModel, criterion, optimizer, device)
    
    train_loader, test_loader = prepare_data(batch_size)
    
    dict = {
                "dataset": "CIFAR-10",
                "epochs": epochs,
                "linear_layers": 3,
                "learning_rate": learning_rate,
                "architecture": "MLP",
                "batch_size": batch_size,
                "conv_layers": 0,
            }
    
    wandb_login(dict)
    
    trained_model = model_class.train(train_loader, test_loader, epochs)


Analyse

### Batch Size

In [ ]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.fc1 = nn.Linear(3072, 720)
        self.fc2 = nn.Linear(720, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Base Model ist 16
#batch_sizes = [8, 16, 64, 256]
batch_sizes = [32]
# Base Model ist 0.01
learning_rate = 0.01
epochs = 50

for batch_size in batch_sizes:
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer
    baseModel = BaseModel()
    optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)
    
    model_class = HelperClass(baseModel, criterion, optimizer, device)
    
    train_loader, test_loader = prepare_data(batch_size)
    
    dict = {
                "dataset": "CIFAR-10",
                "epochs": epochs,
                "linear_layers": 3,
                "learning_rate": learning_rate,
                "architecture": "MLP",
                "batch_size": batch_size,
                "conv_layers": 0,
            }
    
    wandb_login(dict)
    
    trained_model = model_class.train(train_loader, test_loader, epochs)

 ## Model with Convolutional Layers

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #28*28*6
        self.pool = nn.MaxPool2d(2, 2) #14*14*6
        self.fc1 = nn.Linear(1176, 1024) # 1024
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


batch_size = 64
learning_rate = 0.01
epochs = 50

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
cnnBaseModel = CNNModel()
optimizer = optim.SGD(cnnBaseModel.parameters(), lr=learning_rate)

cnn_model_class = HelperClass(cnnBaseModel, criterion, optimizer, device)

train_loader, test_loader = hp.prepare_data(batch_size)

dict = {
            "dataset": "CIFAR-10",
            "epochs": epochs,
            "linear_layers": 3,
            "learning_rate": learning_rate,
            "architecture": "CNN",
            "batch_size": batch_size,
            "conv_layers": 1,
        }

hp.wandb_login(dict)

trained_model = cnn_model_class.train(train_loader, test_loader, epochs)

In [ ]:
summary(cnnBaseModel, (3, 32, 32), batch_size)

## Model with Convolutional Layers and Pooling

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 16)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(384, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

batch_size = 16
learning_rate = 0.001
epochs = 100

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
baseModel = BaseModel().to(device)
optimizer = optim.SGD(baseModel.parameters(), lr=learning_rate)

model = run_model(baseModel, criterion, optimizer, batch_size, epochs, "CNN", 2, 1)